In [1]:
#유하 사이트에서 influencer 정보 긁어오기
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re

#빈 데이터 프레임 생성
df = pd.DataFrame(columns=['category','channel','followers','views','cost'])

In [14]:
#selenium
driver = webdriver.Chrome('chromedriver')
delay =3
driver.implicitly_wait(delay)
driver.get('https://www.youha.info/login/')

userId ="아이디"
userpwd = "패스워드"
driver.find_element_by_name('username').send_keys(userId)
driver.find_element_by_name('password').send_keys(userpwd)
driver.find_element_by_xpath('/html/body/div/form/div[1]/input[2]').click()

driver.get('https://www.youha.info/index/')
html=driver.page_source
soup = BeautifulSoup(html,'html.parser')
category = list()
for i in soup.find('div',class_='categoryMenu showMenu').find_all('a', href=re.compile('^/category/')):
    category.append([i.get('href'),i.text.strip()])

category

[['/category/all/', '전체 채널 보기'],
 ['/category/beauty/', '뷰티 · 패션'],
 ['/category/food/', '먹방'],
 ['/category/bj/', 'BJ · 엔터테이너'],
 ['/category/daily/', 'Vlog · 일상'],
 ['/category/game/', '게임'],
 ['/category/it/', 'IT · 전자기기'],
 ['/category/music/', '음악 · 춤'],
 ['/category/cooking/', '요리 · 베이킹'],
 ['/category/travel/', '여행 · 아웃도어'],
 ['/category/animal/', '동물 · 펫'],
 ['/category/sports/', '스포츠 · 건강'],
 ['/category/movie/', '영화 · 애니'],
 ['/category/car/', '자동차'],
 ['/category/kids/', '키즈 · 플레이'],
 ['/category/education/', '교육'],
 ['/category/politics/', '정치 · 시사'],
 ['/category/fortune/', '운세 · 타로'],
 ['/category/stock/', '부동산 · 투자'],
 ['/category/painting/', '그림 · 페인팅'],
 ['/category/etc/', '기타']]

In [15]:
#필요없는 카테고리 제거
del category[20]
del category[16:19]
del category[9:11]
del category[0]
category

[['/category/beauty/', '뷰티 · 패션'],
 ['/category/food/', '먹방'],
 ['/category/bj/', 'BJ · 엔터테이너'],
 ['/category/daily/', 'Vlog · 일상'],
 ['/category/game/', '게임'],
 ['/category/it/', 'IT · 전자기기'],
 ['/category/music/', '음악 · 춤'],
 ['/category/cooking/', '요리 · 베이킹'],
 ['/category/sports/', '스포츠 · 건강'],
 ['/category/movie/', '영화 · 애니'],
 ['/category/car/', '자동차'],
 ['/category/kids/', '키즈 · 플레이'],
 ['/category/education/', '교육'],
 ['/category/painting/', '그림 · 페인팅']]

In [38]:
idx =0
for cate in category:
    driver.get('https://www.youha.info{0}'.format(cate[0]))
    cate_name = cate[1]
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    count=1
    #페이지 카운팅 copy selector
    for i in soup.select('body > div > div.fit > div > div > a '):
        count +=1
    driver.implicitly_wait(delay)
    #print("count {0}".format(count))
    if count > 30:
        count = 30
    for i in range(2,count-1):
        driver.get('https://www.youha.info{0}?page={1}'.format(cate[0],i))
        driver.implicitly_wait(delay)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        for i in [2,12]:
            channelname =  soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(4) > div > a:nth-child(1) '.format(i))
            followers = soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(5)'.format(i))
            views = soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(7) '.format(i))
            cost =  soup.select('body > div > div.fit > form > div.scroll > table > tbody > tr:nth-child({0}) > td:nth-child(9) '.format(i))
            #print(channelname[0].text.strip())
            #print(followers[0].text.strip())
            #print(views[0].text.strip()+' ' +comments[0].text.strip() +' ' + cost[0].text.strip())
            #데이터 프레임에 row 행 추가
            df.loc[idx] = [cate_name,channelname[0].text.strip(), followers[0].text.strip(), views[0].text.strip(),cost[0].text.strip() ]
            idx +=1
        driver.implicitly_wait(delay+2)
    
df


,category,channel,followers,views,cost
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000"
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000"
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000"
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000"
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000"
...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200"
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200"
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400"
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000"


In [40]:
df.to_csv('data/influencer.csv',index=False)

In [37]:
df = pd.read_csv('data/influencer.csv')
df

,category,channel,followers,views,cost,category_id
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000
...,...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000


In [7]:
import numpy as np

category= pd.read_csv('data/category.csv')
category
df['category_id']  =np.nan

,category_id,category,parent_id,code
0,100000,외식업,0,0
1,101000,한식음식점,100000,0
2,101001,한식/백반/한정식,101000,Q01A01
3,101002,갈비/삼겹살,101000,Q01A02
4,101003,곱창/양구이전문,101000,Q01A03
...,...,...,...,...
274,701016,교복판매,701000,D05A16
275,702000,패션잡화,700000,0
276,702001,섬유/직물/주단,702000,D05A18
277,702002,특수복판매,702000,D05A19


In [24]:

for idx in df.index:
    if(df.iloc[idx]['category']=='뷰티 · 패션'):
        df['category_id'][idx]= '500000'
    elif(df.iloc[idx]['category']=='먹방'):
        df['category_id'][idx]= '100000'
    elif(df.iloc[idx]['category']=='BJ · 엔터테이너'):
        df['category_id'][idx]= '800000'
    elif(df.iloc[idx]['category']=='Vlog · 일상'):
        df['category_id'][idx]= '800000'
    elif(df.iloc[idx]['category']=='게임'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='IT · 전자기기'):
        df['category_id'][idx]= '600000'
    elif(df.iloc[idx]['category']=='음악 · 춤'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='요리 · 베이킹'):
        df['category_id'][idx]= '100000'
    elif(df.iloc[idx]['category']=='스포츠 · 건강'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='영화 · 애니'):
        df['category_id'][idx]= '300000'
    elif(df.iloc[idx]['category']=='자동차'):
        df['category_id'][idx]= '400000'
    elif(df.iloc[idx]['category']=='키즈 · 플레이'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='교육'):
        df['category_id'][idx]= '200000'
    elif(df.iloc[idx]['category']=='그림 · 페인팅'):
        df['category_id'][idx]= '200000'

//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
//anaconda3/envs/recommender/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [39]:
#float to int
print(df.dtypes)
df['category_id']= df['category_id'].astype('int')
df

category       object
channel        object
followers      object
views          object
cost           object
category_id     int64
dtype: object


,category,channel,followers,views,cost,category_id
0,뷰티 · 패션,새벽,"644,000","13,978","30,912,000",500000
1,뷰티 · 패션,DAISY 데이지,"496,000","45,424","23,808,000",500000
2,뷰티 · 패션,딩고 뷰티 / dingo beauty,"424,000","47,901","20,352,000",500000
3,뷰티 · 패션,So Young,"361,000","1,361","17,328,000",500000
4,뷰티 · 패션,유정 YU JEONG,"304,000","19,673","14,592,000",500000
...,...,...,...,...,...,...
581,그림 · 페인팅,픽츄어림,"37,700",331,"603,200",200000
582,그림 · 페인팅,MEATRABBIT육식토끼,"31,200",186,"499,200",200000
583,그림 · 페인팅,KKMI끄미,"26,900",220,"430,400",200000
584,그림 · 페인팅,히됴 Hidyo,"23,000",62,"368,000",200000


In [40]:
df.to_csv('data/influencer.csv',index=False)